#### Faiss
Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

- Install faiss-cpu incase you are using it on local

In [54]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

loader=TextLoader("speech.txt")
documents=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
docs=text_splitter.split_documents(documents)

In [56]:
docs[0].page_content[:100]

'The world must be made safe for democracy. Its peace must be planted upon the tested foundations of '

In [58]:
embeddings=OllamaEmbeddings(model="llama3.2:1b")
db=FAISS.from_documents(docs,embeddings)
db

In [59]:
### querying 
query="How does the speaker describe the desired outcome of the war?"
docs=db.similarity_search(query)
docs[0].page_content[:100]

'We have borne with their present government through all these bitter months because of that friendsh'

#### As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [61]:
retriever=db.as_retriever()
docs=retriever.invoke(query)

In [62]:
docs[0].page_content[:100]

'We have borne with their present government through all these bitter months because of that friendsh'

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [64]:
docs_and_score=db.similarity_search_with_score(query)

In [65]:
docs_and_score[0][0].page_content[:100]

'We have borne with their present government through all these bitter months because of that friendsh'

- We can also pass embeddings to the db for search

In [67]:
# create emd
embedding_vector=embeddings.embed_query(query)

In [68]:
# pass e,bd in to db search
docs_score=db.similarity_search_by_vector(embedding_vector)

In [69]:
docs_score[0].page_content

'We have borne with their present government through all these bitter months because of that friendshipâ€”exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, still have an opportunity to prove that friendship in our daily attitude and actions toward the millions of men and women of German birth and native sympathy who live among us and share our life, and we shall be proud to prove it toward all who are in fact loyal to their neighbors and to the government in the hour of test. They are, most of them, as true and loyal Americans as if they had never known any other fealty or allegiance. They will be prompt to stand with us in rebuking and restraining the few who may be of a different mind and purpose. If there should be disloyalty, it will be dealt with with a firm hand of stern repression; but, if it lifts its head at all, it will lift it only here and there and without countenance except from a lawless and malignant few.'

In [70]:
### Saving And Loading
db.save_local("faiss_index")

In [71]:
new_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)

In [72]:
docs[0].page_content

'We have borne with their present government through all these bitter months because of that friendshipâ€”exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, still have an opportunity to prove that friendship in our daily attitude and actions toward the millions of men and women of German birth and native sympathy who live among us and share our life, and we shall be proud to prove it toward all who are in fact loyal to their neighbors and to the government in the hour of test. They are, most of them, as true and loyal Americans as if they had never known any other fealty or allegiance. They will be prompt to stand with us in rebuking and restraining the few who may be of a different mind and purpose. If there should be disloyalty, it will be dealt with with a firm hand of stern repression; but, if it lifts its head at all, it will lift it only here and there and without countenance except from a lawless and malignant few.'